* creates `deriv/timing_df_recall.csv`
- make and save a dataframe containing onset and offset TRs for each cue presentation during recall

In [6]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt

import sklearn
import brainiak
import nilearn as nl
from nilearn import image, plotting, input_data

from scipy.spatial import distance

pd.options.display.max_rows = 200

- [SUB]_recall.log does not exist for all sub
- use [sub]_recall_mylog.log instead

In [56]:
sub_num = 12
# def read_logfile(sub_num):

fpath = 'data/behav/silvy_buckets/sub%.2iday2/%i_recall_mylog.log'%(100+sub_num,sub_num)
f = open(fpath, "r")
first_line = f.readline()

first_tstamp = float(first_line.split()[1])

for x in f:
  RV,tstamp,stim = x.split(' ')
  stim = stim.split('/')[-1].split('.')[0][1:]

  print(RV)

IndentationError: unexpected indent (<ipython-input-56-31bd2dffad0a>, line 3)

In [7]:
def read_logfile(sub_num,task='recall'):
  """ 
  read psychopy logifle into dataframe
  """
  # initialize dataframe
  df = pd.DataFrame(columns=['tstamp','logdata'])
  df = df.astype({'tstamp':float})
  # open logfile
  fpath = 'data/behav/silvy_buckets/sub%iday2/%i_%s.log'%(100+sub_num,sub_num,task)
  f = open(fpath, "r")
  # loop over logfile rows
  for x in f:
    tstamp,B,C = x.split('\t')
    tstamp = float(tstamp)
#     df.loc[tstamp,'logtype'] = B[:-1]
    df.loc[tstamp,'logdata'] = C[:-1]
    df.loc[tstamp,'tstamp'] = tstamp
  return df

def reindex_and_TR_column(df):
  TR_rate = 1.5
  # first TR
  first_TR_tstamp = df[df.logdata=='Keypress: equal'].iloc[0].tstamp
  # center tstamps on first TR
  df.tstamp = df.tstamp-first_TR_tstamp
  # include TR column
  df['TR'] = np.ceil(df.tstamp/TR_rate)
  df = df.astype({'TR':int})
  # remove negative TRs
  df = df[df.TR>-1]
  # reindex by row number
  df.index = np.arange(len(df))
  return df

def include_wed_cols(df):
  ''' includes columns with wed_num and wed_id 
  '''
  df.loc[:,'wed_num'] = 99
  df.loc[:,'wed_id'] = 99
  wed_start_idx = df[df.logdata.str[:len("New trial")]=="New trial"].index
  wed_id_series = df.iloc[wed_start_idx].logdata.str.split('/').str[-1].str.split('.').str[0].str[1:]
  for wed_num,(t0,t1,wed_id) in enumerate(zip(wed_start_idx[:-1],wed_start_idx[1:],wed_id_series)):
    df.loc[t0:t1,'wed_num']=wed_num
    df.loc[t0:t1,'wed_id']=wed_id
  df.loc[wed_start_idx[-1]:,'wed_num']=11
  df.loc[wed_start_idx[-1]:,'wed_id']=wed_id_series.iloc[-1]
  return df

In [15]:
for sub_n in range(30,35):
  df = read_logfile(sub_n)
  df = reindex_and_TR_column(df)
  df = include_wed_cols(df)
  print(df.shape)
  df.iloc[:200]

(392, 5)
(294, 5)
(249, 5)
(562, 5)
(546, 5)


In [16]:
sub_n=32
df = read_logfile(sub_n)
df = reindex_and_TR_column(df)
df = include_wed_cols(df)
print(df.shape)
df.iloc[:200]

(249, 5)


,tstamp,logdata,TR,wed_num,wed_id
0,0.0000,Keypress: equal,0,99,99
1,0.0110,"Imported recall.csv as conditions, 12 conditio...",1,99,99
2,0.0120,"Created sequence: random, trialTypes=12, nReps...",1,99,99
3,0.0124,"New trial (rep=0, index=0): OrderedDict([('cue...",1,0,1
4,0.0480,text: autoDraw = True,1,0,1
5,1.4995,Keypress: equal,1,0,1
6,2.9990,Keypress: equal,2,0,1
7,4.4990,Keypress: equal,3,0,1
8,5.9990,Keypress: equal,4,0,1
9,7.4989,Keypress: equal,5,0,1


In [18]:
read_logfile(32,task='recall').iloc[:200]

,tstamp,logdata
1.8353,1.8353,Import Error: No module named 'psychtoolbox'. ...
4.8882,4.8882,User requested fullscreen with size [2560 1440...
5.5324,5.5324,window1: recordFrameIntervals = False
5.6983,5.6983,window1: recordFrameIntervals = True
5.8818,5.8818,window1: recordFrameIntervals = False
6.0052,6.0052,Created text_2 = TextStim(__class__=<class 'ps...
6.0140,6.0140,Created waiting = TextStim(__class__=<class 'p...
6.0206,6.0206,Created cueImage = ImageStim(__class__=<class ...
6.0540,6.0540,Created text = TextStim(__class__=<class 'psyc...
6.0993,6.0993,text_2: autoDraw = True
